In [39]:
class Token:       
    def __init__(self, _name = ""):
        self.balances = {}
        self.totalSupply = 0
        self._name = _name
    
    def mint(self, address, amount):
        assert(amount > 0)
        self.balances[address] = self.balances.get(address, 0) + amount
        self.totalSupply += amount
        return True
        
    def burn(self, address, amount):
        assert(amount > 0)
        if self.balances[address] < amount:
            return False
        self.balances[address] = self.balances.get(address, 0) - amount
        self.totalSupply -= amount
    
    def balanceOf(self, address):
        return self.balances.get(address, 0)
    
    def name(self):
        return self._name
    
    def transferFrom(self, sender, recipient, amount):
        assert(amount >= 0)
        if self.balanceOf(sender) >= amount:
            self.balances[sender] = self.balances.get(sender, 0) - amount
            self.balances[recipient] = self.balances.get(recipient, 0) + amount    
            return True
        else:
            return False

In [40]:
token = Token()
token.mint("A", 5)
assert(token.balanceOf("A") == 5)

token.transferFrom("A", "B", 3)
assert(token.balanceOf("B") == 3)
assert(token.balanceOf("A") == 2)

token.transferFrom("A", "C", 2)
assert(token.balanceOf("A") == 0)
assert(token.balanceOf("C") == 2)

In [41]:
import time 
import decimal
from decimal import Decimal 
math = decimal.Context()

# class InterestToken
#  __init__(rate, _rateAccumulator)
#  balanceOfUnderlying(address)
#  accrueInterest()
#  updateRate(update)
#  yearlyRate()

class blockchain:
    
    def __init__(self, _timestamp, _block):
        self.timestamp = _timestamp
        self.block = _block
    
    def incrementBlock(self):
        self.block += 1
        self.timestamp += 15
    
    def updateTimestamp(self, _timestamp):
        self.timestamp = _timestamp
    
    def now(self):
        return self.timestamp


class InterestToken(Token):
    # rate is a yearly rate, converted to a rate compounded per second
    def __init__(self, _name, _blockchain, rate, _rateAccumulator):
        Token.__init__(self, _name)
        self.blockchain = _blockchain
        self.lastUpdate = self.blockchain.now()
        self.rateAccumulator = _rateAccumulator
        # 31622400 is the number of seconds in a year
        self.rate = math.power(1 + Decimal(rate), 1/Decimal(31622400))  
    
    def balanceOfUnderlying(self, address):
        return self.balanceOf(address) * self.rateAccumulator
    
    def transferUnderlying(self, sender, recipient, amount):
        value = amount / self.rateAccumulator
        self.transferFrom(sender, recipient, value)
        
    def accrueInterest(self):
        now = self.blockchain.now()
        if now > self.lastUpdate:
            total_time =  now - self.lastUpdate
            self.rateAccumulator = math.power(self.rate, total_time) * self.rateAccumulator
            self.lastUpdate = now
    
    def updateRate(self, update):
        newRate = math.power(1 + Decimal(update), 1/Decimal(31622400)) - 1
        self.rate = self.rate + newRate
        if self.rate < Decimal(1):
            self.rate = Decimal(1)
    
    def yearlyRate(self):
        return math.power(self.rate, Decimal(31622400)) - 1
    
    def mintInUnderlying(self, address, amount):
        self.mint(address, amount/self.rateAccumulator)
        
    def burnInUnderlying(self, address, amount):
        self.burn(address, amount/self.rateAccumulator)

In [42]:
chain = blockchain(0,1)
token = InterestToken("Token", chain, .02, 1)
token.mint("A", 1000)
#token.lastUpdate = token.lastUpdate - 31622400
chain.updateTimestamp(31622400)
token.accrueInterest()
token.balanceOfUnderlying("A")
assert(int(token.balanceOfUnderlying("A")) == 1020)

In [43]:
chain = blockchain(0,1)
token = InterestToken("Token", chain, .02, 1)
token.mint("A", 1000)
#token.lastUpdate = token.lastUpdate - 31622400
# Note that the per second rates are multiplied, not added
token.updateRate(.001)
token.updateRate(.001)
token.updateRate(-.001)
token.updateRate(.001)
token.updateRate(.001)
#print(token.yearlyRate())
chain.updateTimestamp(31622400)
print(token.rate)
token.accrueInterest()
assert(int(token.balanceOfUnderlying("A")) == 1023)

1.000000000721011950427853002


In [44]:
chain = blockchain(1,1)
uToken = InterestToken("Token", chain, .04, 1)
for x in range(10000):
    chain.incrementBlock()
    uToken.accrueInterest()

uToken.rateAccumulator

Decimal('1.000186059700104401460770319')

In [188]:
class QueueExchange(Token):
    def __init__(self, _token, _uToken : Token , _sToken : InterestToken):
        super().__init__(_token)
        self.uToken = _uToken
        self.sToken = _sToken
        self.underlying = 0
        self.synthetic  = 0
        self.address = "queueball"
        self.book = {}
        self.uTokenBalance = 0
        self.sTokenBalance = 0
        self.sTokenBalanceUnderlying = 0
        self.conversionRate = Decimal('1')
        self.savedRateAccumulator = self.sToken.rateAccumulator
        self.fee = Decimal('0.0005')
        
    # This proxy function lets us add logic to transfers if needed later
    def transferToken(self, token, sender, recipient, amount):
        if token == self.sToken:
            return token.transferUnderlying(sender, recipient, amount)
        else:
            return token.transferFrom(sender, recipient, amount)
    
    # Before changing the self.totalSupply of the Liquidity Provisioning shares, 
    # we must make sure we update the self.conversionRate 
    # self.totalSupply is not changed by adding accumulated interest
    def collectSyntheticInterest(self):
        if self.totalSupply == 0:
            return True
        # calculate interest earned
        interestEarned = self.sTokenBalance * (self.sToken.rateAccumulator - self.savedRateAccumulator)
        # interest is shared across all users
        newTotalValue = (self.totalSupply * self.conversionRate)  + interestEarned
        self.conversionRate = newTotalValue / self.totalSupply 
        # update 
        self.savedRateAccumulator = self.sToken.rateAccumulator
        self.sTokenBalanceUnderlying = self.sTokenBalanceUnderlying + interestEarned
        # fail if our accounting of balance does not match true balance
        # this cannot be done in Solidity as transfers can be sent without contracts consent
        assert(self.sTokenBalanceUnderlying == self.sToken.balanceOfUnderlying(self.address))
    
    def depositUnderlying(self, address, amount):
        if amount > 0:
            # before updating, collect interest
            self.collectSyntheticInterest()
            self.transferToken(self.uToken, address, self.address, amount)
            self.uTokenBalance += amount
            self.balances[address] = self.balances.get(address, 0) + amount/self.conversionRate
            self.totalSupply += amount/self.conversionRate
            return True
        else:
            return False
        
    def depositSynthetic(self, address, amount):
        if amount > 0:        
            # before updating, collect interest
            self.collectSyntheticInterest()
            #Transfer synthetic
            self.transferToken(self.sToken, address, self.address, amount)
            self.sTokenBalance += amount / self.sToken.rateAccumulator
            self.sTokenBalanceUnderlying += amount
            self.balances[address] = self.balances.get(address, 0) + amount/self.conversionRate
            self.totalSupply += amount/self.conversionRate
            return True
        else:
            return False
    
    # withdraw underlying token using underlying token as unit of account 
    def withdrawUnderlying(self, address, amount):
        if amount > 0:
            # before updating, collect interest
            self.collectSyntheticInterest()
            #check if funds are sufficient
            if amount > self.balances.get(address, 0) * self.conversionRate:
                return False
            # Is there enough balance to send?
            if amount > self.uTokenBalance:
                return False
            self.balances[address] -= amount/self.conversionRate
            # when withdrawing underlying, a fee is taken
            fee = amount * self.fee
            amountMinusFee = amount - fee
            newTotalValue = (self.totalSupply * self.conversionRate)  - amountMinusFee
            self.totalSupply -= amount/self.conversionRate
            self.uTokenBalance -= amountMinusFee
            self.conversionRate = newTotalValue / self.totalSupply 
            return self.transferToken(self.uToken, self.address, address, amountMinusFee)
        
    # withdraw synthetic token using underlying token as unit of account
    def withdrawSynthetic(self, address, amount):
        if amount > 0:
            # before updating, collect interest
            self.collectSyntheticInterest()
            #check if funds are sufficient
            if amount > self.balances.get(address, 0) * self.conversionRate:
                return False
            # Is there enough balance to send?
            if amount > self.sTokenBalance:
                return False
            self.balances[address] -= amount/self.conversionRate
            self.totalSupply -= amount/self.conversionRate
            self.sTokenBalance -= amount / self.sToken.rateAccumulator
            self.sTokenBalanceUnderlying -= amount
            return self.transferToken(self.sToken, self.address, address, amount)
    
    
    def measureImbalance(self):
        self.collectSyntheticInterest()
        # If balance difference is under 5% of the total, treat as if there is no imbalance 
        if abs((self.uTokenBalance - self.sTokenBalanceUnderlying) / (self.totalSupply * self.conversionRate)) < Decimal('0.5'):
            return Decimal('0')
        return self.uTokenBalance - self.sTokenBalanceUnderlying
    
        

In [189]:
chain = blockchain(1,1)
uToken = Token("Token")
uToken.mint("A", Decimal('1000'))
sToken = InterestToken("pyToken", chain, Decimal('.04'), Decimal('1'))
sToken.mint("B", Decimal('1000'))
qx = QueueExchange("QX", uToken,sToken)
qx.depositUnderlying("A", Decimal('1000'))
qx.depositSynthetic("B", Decimal('1000'))
chain.updateTimestamp(31622400)
sToken.accrueInterest()
qx.collectSyntheticInterest()
# Half underlying, Half pyToken means half the total interest rate
assert(qx.conversionRate == Decimal('1.019999999355053037491798033'))
qx.withdrawUnderlying("B", 500)
assert(qx.conversionRate == Decimal('1.020165583770671446317224349'))
qx.withdrawSynthetic("A", 500)
chain.updateTimestamp(2 * 31622400)
sToken.accrueInterest()
qx.collectSyntheticInterest()
assert(uToken.balanceOf("queueball") + sToken.balanceOfUnderlying("queueball") - \
       (qx.conversionRate * qx.totalSupply) < Decimal('0.000000000000000001'))

In [235]:

class Oracle:
    def __init__(self):
        self.price = 0 
    
    def currentPrice(self):
        return self.price
    
    def setPrice(self, price):
        self.price = price


class pyToken (InterestToken):
    def __init__(self, _blockchain, _underlying : Token, _oracle):  
        super().__init__("", _blockchain, Decimal('0'), Decimal('1'))
        self.underlying = _underlying
        self.name = "py" + self.underlying.name()
        self.cdp = {self.underlying.name(): {}}
        self.tokens = {}
        self.address = self.name
        # oracle for the pyToken <> pyToken pair
        self.oracle = _oracle
        self.qx = 0
        self.interestUpdateAmount = Decimal('0.0001')
        self.collateralizationRatio = Decimal('1.5')
        self.debtAccumulator = self.rateAccumulator
        self.debtRate = self.rate
        
    # Admin Functions
    def setPair(self, _pairedpyToken : InterestToken):
        self.pairedpyToken = _pairedpyToken
        self.validCollateral.append(self.pairedpyToken.name())    
        self.tokens[self.pairedpyToken.name()] = self.pairedpyToken
        self.cdp[self.pairedpyToken.name()] = {}
        
    def setQueueExchange(self,exchange):
        self.qx = exchange
        
    def addCollateralType(self, token, oracle):
        self.tokens[token.name()] = { 'token':token, 'oracle':oracle }
        self.cdp[token.name()] = {}
    
    # User Functions
    def addCollateral(self, token, address, amount):
        if token not in self.tokens.keys():
            return False
        result = self.tokens[token]['token'].transferFrom(address, self.address, amount)
        if result:
            self.cdp[token][address] = {'amount':amount, 'debt':Decimal('0')}
        else:
            return False
      
    def withdrawCollateral(self, token, address, amount):
        if token not in self.tokens.keys():
            return False
        # check oracle price
        price = self.oracle.currentPrice()
        minimumCollateral = self.oracle.currentPrice() * self.collateralizationRatio * \
                            (self.cdp[token][address]['debt'] * self.debtAccumulator - amount)
        if self.cdp[token][address]['amount'] < minimumCollateral:
            return False
        result = self.tokens[token].transferFrom(self.address, address, amount)
        if result:
            self.cdp[token][address]['amount'] = self.cdp[token][address]['amount'] - amount
    
    # accrue for both debt and interest
    def accrueInterest(self):
        now = self.blockchain.now()
        if now > self.lastUpdate:
            total_time =  now - self.lastUpdate
            self.rateAccumulator = math.power(self.rate, total_time) * self.rateAccumulator
            self.debtAccumulator = math.power(self.debtRate, total_time) * self.debtAccumulator
            self.lastUpdate = now
        
    # updateRate for both debt and interest
    # this contract implicity assumes that this is called every block
    def updateRate(self, update):
        newRate = math.power(1 + Decimal(update), 1/Decimal(31622400)) - 1
        self.rate = self.rate + newRate
        self.debtRate = self.rate + newRate
        if self.rate < Decimal(1):
            self.rate = Decimal(1)
        
    # borrow against a particular tokens worth of collateral
    def borrow(self, token, address, amount):
        if token not in self.tokens.keys():
            return False
        if amount < 0:
            return False
        price = self.oracle.currentPrice()
        minimumCollateral = self.oracle.currentPrice() * self.collateralizationRatio * \
                            (self.cdp[token][address]['debt'] * self.debtAccumulator + amount)
        if self.cdp[token][address]['amount'] < minimumCollateral:
            return False
        # Add debt
        self.cdp[token][address]['debt'] = self.cdp[token][address]['debt'] + Decimal(amount) / self.debtAccumulator
        # Add borrow
        self.mintInUnderlying(address, amount)
        
        
    def position(self, token, address):
        return self.cdp[token][address]
    
    def repay(self, token, address, amount):
        if token not in self.tokens.keys():
            return False
        if amount < 0:
            return False
        
    def updateRates(self):
        imbalance = qx.measureImbalance()
        if imbalance == 0:
            return 
        if imbalance > 0:
            # lower interest rate
            self.updateRate(-self.interestUpdateAmount)
        elif imbalance < 0:
            # raise interest rate
            self.updateRate(self.interestUpdateAmount)
    
        
        
    
    

In [191]:
chain = blockchain(1,1)
oracleA = Oracle()
tokenA = Token("TokenA")
collateral = Token("Collateral")
oracleC = Oracle()
pyA = pyToken(chain, tokenA, oracleA)
pyA.addCollateralType(collateral, oracleC)
qx = QueueExchange("QX:TokenA", tokenA , pyA)
pyA.setQueueExchange(qx)

oracleC.setPrice(Decimal('2'))
tokenA.mint("A", Decimal('1000'))
collateral.mint("B", Decimal('1000'))
pyA.addCollateral("Collateral", "B", Decimal('100'))
pyA.borrow("Collateral", "B", Decimal('10'))
assert(pyA.balanceOf("B") == Decimal('10'))
qx.depositSynthetic("B", Decimal('10'))
qx.depositUnderlying("A", Decimal('10'))
qx.withdrawSynthetic(self, address, amount)


True

In [192]:
chain.incrementBlock()
pyA.updateRates()
pyA.accrueInterest()

In [193]:
pyA.rate

Decimal('1.000000000003162157215564487')

In [194]:
(qx.uTokenBalance - qx.sTokenBalanceUnderlying) / (qx.totalSupply * qx.conversionRate)
qx.collectSyntheticInterest()
qx.sTokenBalanceUnderlying

Decimal('10.00000000047432358234517225')

In [195]:
qx.totalSupply * qx.conversionRate

Decimal('10.00000000047432358234517225')

In [ ]:

class Interface:
    
    def __init__(self, _chain : blockchain, _uTokenA : Token, _uTokenB : Token, oracle):
        self.A = _uTokenA
        self.B = _uToken
        self.chain = _chain
        self.oracle = oracle
        self.pyA = pyToken(_chain, self.A, oracle)
        self.pyB = pyToken(_chain, self.B, oracle)
        self.pyA.setPair(self.pyB)
        self.pyB.setPair(self.pyA)
        self.qxA = QueueExchange("QX:" + self.A.name, self.A , self.pyA)
        self.qxB = QueueExchange("QX:" + self.B.name, self.B , self.pyB)
        self.pyA.setQueueExchange(self.pyB)
        self.pyB.setQueueExchange(self.pyA)
        # Add "ETH" as acceptable collateral
        self.eth = Token("ETH")
        self.ethAOracle = Oracle()
        self.ethBOracle = Oracle()
        self.ethAOracle.setPrice(Decimal('100'))
        self.ethBOracle.setPrice(Decimal(100/oracle.currentPrice()))
        self.pyA.addCollateralType(self.eth, self.ethOracle)
        self.pyB.addCollateralType(self.eth, self.ethOracle)
        
    def updateInterestRates(self):
        self.pyA.updateRates()
        self.pyB.updateRates()
        self.pyA.accrueInterest()
        self.pyB.accrueInterest()
    
    def eachBlock(self):
        self.chain.incrementBlock()
        self.updateInterestRates()
        

# External market that provides "infinite" liquidity
def ReferenceExchange:
    def __init__():
        tokenPairs = []
    
    def addPair(tokenA, tokenB):
        # mint a lot of tokens for both sides
        pass
        # record pair
    
    def updatePairPrice(tokenA, tokenB, price):
        pass
    
    def trade(self, tokenA, tokenB, tokenAAmount, address):
        # transfer in tokenA
        pass
        # calculate amount of tokenB to send
        
        # send tokenB to address
        


In [236]:
import numpy 


    
class Agent:
    def __init__(self, _interface):
        self.interface = _interface
        self.address = numpy.random.randint(1,100000000000000000)
        
    def setup(self):
        pass
    
    def performActions(self):
        pass
    
    def shutdown(self):
        pass
    
    
# Yield agents sell their underlying for pyTokens and hodl them
class YieldAgent(Agent):
    def __init__(self, _interface):
        super().__init__(_interface)
        
        
    def setup(self):   
        self.initialA = min(Decimal(round(numpy.random.lognormal(14, 4))), Decimal(1000000000))
        self.initialB = min(Decimal(round(numpy.random.lognormal(14, 4))), Decimal(1000000000))
        self.interface.A.mint(self.address, Decimal(initialA))
        self.interface.B.mint(self.address, Decimal(initialB))
        self.initial_exchange_rate = self.interface.oracle.currentPrice()
    
    def performActions(self):
        amount = min(self.interface.qxA.uTokenBalance, self.interface.A.balanceOf())
        self.interface.qxA.depositUnderlying(self.address, amount)
        self.interface.qxA.withdrawSynthetic(self.address, self.interface.qxA.balanceOf())
        
        amount = min(self.interface.qxB.uTokenBalance, self.interface.B.balanceOf())
        self.interface.qxB.depositUnderlying(self.address, amount)
        self.interface.qxB.withdrawSynthetic(self.address, self.interface.qxB.balanceOf())
    
    
    def shutdown(self):
        pass
    
    
    
# Borrow agents level up on ETH collateral by selling pyTokens
class BorrowAgent(Agent):
    def __init__(self, _interface):
        super().__init__(_interface)
        
    def setup(self):
        self.initialETH = min(Decimal(round(numpy.random.lognormal(14, 4))), Decimal(1000000000))
        self.interface.eth.mint(self.address, Decimal(initialA))
    
    def ETHForA(self):
        pos = self.interface.pyA.position("ETH", self.address)
        maxborrowable = (pos['amount'] * self.ethAOracle.currentPrice() - pos['debt']) / (self.interface.A.collateralizationRatio )
        maxavailable = min(self.interface.qxA.uTokenBalance, maxborrowable)
        if maxavailable > 0:
            self.interface.pyA.borrow("ETH", self.address, maxavailable)
            self.interface.qxA.depositSynthetic(self.address, maxavailable)
            self.interface.qxA.withdrawUnderlying(self.address, maxavailable)
            #Trade underlying for ETH and deposit
            
    def ETHForB(self):
        balance = self.interface.eth.balanceOf()
        if balance > 0:
    
    def performActions(self):
        start = random.choice(["A","B"])
        if start == "A":
            ETHForA()
            ETHForB()
        else:
            ETHForB()
            ETHForA()
        
    
    def shutdown(self):
        pass
    
    
chain = blockchain(1,1)
oracleA = Oracle()
tokenA = Token("TokenA")
collateral = Token("Collateral")
oracleC = Oracle()
pyA = pyToken(chain, tokenA, oracleA)
pyA.addCollateralType(collateral, oracleC)
qx = QueueExchange("QX:TokenA", tokenA , pyA)
pyA.setQueueExchange(qx)

oracleC.setPrice(Decimal('2'))
tokenA.mint("A", Decimal('1000'))
collateral.mint("B", Decimal('1000'))
pyA.addCollateral("Collateral", "B", Decimal('100'))
pyA.borrow("Collateral", "B", Decimal('10'))
assert(pyA.balanceOf("B") == Decimal('10'))
qx.depositSynthetic("B", Decimal('10'))
qx.depositUnderlying("A", Decimal('10'))
qx.withdrawSynthetic(self, address, amount)

IndentationError: expected an indented block (<ipython-input-236-8c739f8cb448>, line 66)

In [14]:
import numpy

chain = blockchain(1,1)

chain = blockchain(1,1)
oracle = Oracle()
oracle.setPrice(Decimal('2'))
tokenA = Token("TokenA")
tokenB = Token("TokenB")
interface = Interface(chain, tokenA, tokenB, oracle)

#Create users
user = Agent(interface)

user.setup()

for block in range(10000):
    # perform interest rate update at beginning of block
    interface.eachBlock()
    
    user.performActions()
    

    
user.shutdown()

TypeError: __init__() missing 2 required positional arguments: '_underlying' and '_oracle'

In [234]:
import numpy


Decimal('345783')